<a href="https://colab.research.google.com/github/zainiba/DigitalMediaBot/blob/main/condensetrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import tensorflow as tf

import numpy as np
import os
import time

In [71]:
# Read, then decode for py2 compat.
text = open('input1.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
vocab = sorted(set(text))


In [72]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [73]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

I
t
 
s
h


In [74]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)



In [76]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [77]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [78]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [79]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [80]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [81]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 98) # (batch_size, sequence_length, vocab_size)


In [83]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([22, 60, 33, 59, 24, 27, 12, 49, 55, 97, 45, 92, 49, 55, 35, 75, 21,
       64, 27, 28, 64, 28, 20, 94, 10, 62, 70, 46, 37,  1, 36, 67, 25, 29,
       82, 15, 25, 44, 79, 87, 73, 29, 46, 25, 32, 97, 22, 67, 41, 35, 46,
       87, 59, 28, 55, 47, 41, 67, 67, 88, 54,  6, 88, 55, 95, 36, 12, 32,
        6,  4, 63, 91, 42, 91, 28, 20,  5, 56, 44, 75, 25, 39, 52, 69, 77,
       83, 28, 77, 81, 13, 39,  5, 45, 94, 30, 49, 92, 41, 83, 48])

In [84]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 98)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.586711


In [85]:
model.compile(optimizer='adam', loss=loss)

In [86]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [89]:
EPOCHS = 60


In [90]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
40/40 [==============================] - 167s 4s/step - loss: 3.0787
Epoch 2/60
40/40 [==============================] - 167s 4s/step - loss: 2.5354
Epoch 3/60
40/40 [==============================] - 165s 4s/step - loss: 2.3713
Epoch 4/60
40/40 [==============================] - 165s 4s/step - loss: 2.2574
Epoch 5/60
40/40 [==============================] - 166s 4s/step - loss: 2.1391
Epoch 6/60
40/40 [==============================] - 165s 4s/step - loss: 2.0216
Epoch 7/60
40/40 [==============================] - 165s 4s/step - loss: 1.9135
Epoch 8/60
40/40 [==============================] - 164s 4s/step - loss: 1.8044
Epoch 9/60
40/40 [==============================] - 168s 4s/step - loss: 1.7079
Epoch 10/60
40/40 [==============================] - 169s 4s/step - loss: 1.6217
Epoch 11/60
40/40 [==============================] - 172s 4s/step - loss: 1.5428
Epoch 12/60
40/40 [==============================] - 168s 4s/step - loss: 1.4778
Epoch 13/60
40/40 [==================

In [91]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_60'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([64, None]))

In [107]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.7

    # Here batch size == 1
    # model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [111]:
print(generate_text(model, start_string=u"Technology "))

Technology in and of its inction is made between reading, watching, playing, and  experiencing  digital fiction is rarer than access to print picnic table I described would not be designed, a biology produce fantastical and elaborate hallucinations of ornamental patterns when gazing.  The reproduction and redistribution of digital control and design of this potround us, back to our needs to explore new worlds with other people, born in a plant has the instructions or algorithms are required for the libraries6 of the future as well as purveyors of creativity is altered the function of the event score is "a ciece. I may established ideas of space, form, function, and use in order to reflection place in San Jose, California. A symposium kicked off the Biennial. The to resulted a string of eights. Eight is a guide, and cutting the pattern s pieces out by hand with die-cut leaf tissue s disembodied cells contain such total potentiality, but imagining how this technological predisposition is